In [138]:
import os
import re
import pandas as pd
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

In [139]:
folder = Path('../sheets-cleanup/processed_sheets/')
files = list(folder.glob('*.xlsx'))

print(f'Total de arquivos: {len(files)}')

Total de arquivos: 184


In [140]:
load_dotenv()
client = OpenAI()


PROMPT_TEMPLATE = """
Você é um especialista sênior em classificação fiscal brasileira (NCM), com domínio da TIPI, das Regras Gerais de Interpretação (RGI 1 a 6) e das Notas Explicativas do Sistema Harmonizado.

Sua tarefa é determinar o NCM correto e vigente (8 dígitos) com base exclusivamente no produto descrito.

IMPORTANTE SOBRE O FORMATO DA DESCRIÇÃO:

As descrições serão uma única linha contendo:

NOME REAL DO PRODUTO + MARCAS + MODELOS DE MOTOCICLETA + ANOS + CÓDIGO INTERNO

Exemplo:
RETENTOR PINHAO IRON BIZ100 POP100 DREA POP110 SHINERAY 50 041640

REGRAS OBRIGATÓRIAS:

1. O NOME REAL DO PRODUTO sempre estará no início da descrição.
2. Após o nome do produto, podem aparecer:
   - marcas
   - modelos de motocicletas
   - cilindradas
   - anos
   - códigos internos numéricos
3. Ignore completamente:
   - marcas comerciais
   - modelos de moto
   - anos
   - números isolados
   - códigos internos
4. Classifique exclusivamente pela natureza física e função do produto.
5. Não utilize compatibilidade com veículos como critério principal de classificação.
6. Não presuma características técnicas que não estejam implícitas no nome do produto.
7. Se houver ambiguidade relevante que impeça classificação segura, retorne apenas: INDETERMINADO
8. O código deve conter exatamente 8 dígitos numéricos válidos na NCM vigente.

FORMATO DA RESPOSTA:
- Retorne apenas o código NCM com 8 dígitos.
- Não inclua explicações.
- Não inclua texto adicional.
- Não inclua pontuação.

Descrição do produto:
{description}
"""
NCM_COLUMN = 'NCM'

In [141]:
def get_ncm(description: str) -> str:
    prompt = PROMPT_TEMPLATE.format(description=description)

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        temperature=0,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )

    raw_text = response.choices[0].message.content.strip()

    # Extrai apenas 8 dígitos
    match = re.search(r"\b\d{8}\b", raw_text)
    if match:
        return match.group(0)

    return "None"

In [149]:
def process_sheets():
    print(f"Total de arquivos encontrados: {len(files)}\n")

    df_final = pd.DataFrame()

    for file_path in files:
        print(f"Processando arquivo: {file_path.name}")

        df = pd.read_excel(file_path, usecols=[1, 2, 3, 4], dtype=str, index_col=False)

        if df.empty:
            print("Planilha vazia.\n")
            break

        if df.shape[1] < 2:
            print("Coluna descrição não encontrada.\n")
            break

        first_description = str(df.iloc[0, 1]).strip()

        if not first_description or first_description.lower() == "nan":
            print("Descrição não encontrada.\n")
            break

        print(f"Descrição encontrada: {first_description}")

        ncm_code = get_ncm(first_description)

        print(f"NCM encontrado: {ncm_code}")

        df[NCM_COLUMN] = ncm_code
        df_final = pd.concat([df_final, df], ignore_index=True)

        print(f"Atualizado: {file_path.name}\n")

    print('Finalizado')
    return df_final


In [150]:
processed_df = process_sheets()

Total de arquivos encontrados: 184

Processando arquivo: SENSOR OXIGÊNIO.xlsx
Descrição encontrada: SENSOR OXIGENIO IRON CB300 2010 A 2015 201614
NCM encontrado: 90318000
Atualizado: SENSOR OXIGÊNIO.xlsx

Processando arquivo: SUPORTE CELULAR 13_02.xlsx
Descrição encontrada: SUPORTE CELULAR LEHMOX PRETO 2259
NCM encontrado: 39269090
Atualizado: SUPORTE CELULAR 13_02.xlsx

Processando arquivo: SILICONE 15_02_.xlsx
Descrição encontrada: SILICONE ACETICO CINZA WMAX
NCM encontrado: 32089000
Atualizado: SILICONE 15_02_.xlsx

Processando arquivo: PEDAL FREIO - 01_02.xlsx
Descrição encontrada: PEDAL FREIO PROTORK BIZ
NCM encontrado: 87149000
Atualizado: PEDAL FREIO - 01_02.xlsx

Processando arquivo: KIT CILINDRO - 19_01 M.xlsx
Descrição encontrada: KIT CILINDRO RIIE CB300
NCM encontrado: 84099100
Atualizado: KIT CILINDRO - 19_01 M.xlsx

Processando arquivo: TRANSMISSÃO PRIMÁRIA.xlsx
Descrição encontrada: TRANSMISSAO PRIMARIA COMPLETA MHX BURGMAN125 2011 A 2019
NCM encontrado: 87149000
Atualiza

In [151]:
processed_df.head()

,CÓD. BARRA,DESCRIÇÃO,QUANTIDADE,PREÇO,NCM
0,070341180129,SENSOR OXIGENIO IRON CB300 2010 A 2015 201614,2,210,90318000
1,7909201087694,SENSOR OXIGENIO CONDOR SCOOTER NMAX160 2017 A ...,1,NaN,90318000
2,7909201058670,SENSOR OXIGENIO CONDOR NEO115 1210385,2,60,90318000
3,6920240718036,SUPORTE CELULAR LEHMOX PRETO 2259,5,NaN,39269090
4,6920240718036,SUPORTE CELULAR LEHMOX PRATEADO 2259,10,NaN,39269090


In [152]:
processed_df.to_excel('../sanitization/products_with_ncm.xlsx', index=False)